In [12]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
    
root_dir=os.getcwd()
output_folder='step4_plot_summa_outputv3'
outputfile = 'summa_hru.png'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

sim_file='../output/gmet_3hr/output_output_timestep.nc'
# sim_file='../output/gmet_3hr/output_output_day.nc'

var_names = ['pptrate_mean', 'airtemp_mean', 'scalarCanopyWat', 'scalarSWE', 'scalarAquiferStorage', 'scalarTotalSoilWat',
            'scalarSurfaceRunoff_mean', 'scalarSoilBaseflow_mean', 'scalarAquiferBaseflow_mean', 
             'scalarTotalET_mean', 'scalarTotalRunoff_mean']
var_units = ['kg m-2 s-1', 'K', 'kg m-2', 'kg m-2','m', 'kg m-2', 'm s-1', 'm s-1', 'm s-1', 'kg m-2 s-1', 'm s-1']
t_vname = 'time'

hru_indexs = [0, 1, 2]
time_format='%Y-%m-%d %H:%M:%S'
utc_local_shift = -6 #hr shift from UTC to local

time_step = 3 #hr
time_num_perday = 24/time_step

plot_start_date_summary=['2007-10-01 00:00:00']
plot_end_date_summary = ['2017-09-30 00:00:00']

# read simulated flow (m^3/s)
f = xr.open_dataset(sim_file)
var_list = f.variables
sim_time = f[t_vname].dt.strftime(time_format).values[:]

for hru_index in hru_indexs:
    print('hru '+str(hru_index+1))
    with PdfPages(os.path.join(root_dir, output_folder, 
                               outputfile.split('.')[0]+str(hru_index+1)+'.'+outputfile.split('.')[1])) as pdf:    
        for m in range(len(plot_start_date_summary)):

            plot_start_str=plot_start_date_summary[m]
            plot_end_str=plot_end_date_summary[m]
            print(plot_start_str)

            plot_start=int(((datetime.datetime.strptime(plot_start_str,time_format)-datetime.datetime.strptime(sim_time[0],time_format)).days)*time_num_perday)
            plot_len=int(((datetime.datetime.strptime(plot_end_str,time_format)-datetime.datetime.strptime(plot_start_str,time_format)).days)*time_num_perday)

            plot_start_date=datetime.datetime.strptime(plot_start_str,time_format)
            t=[plot_start_date+datetime.timedelta(hours=hr*time_step+utc_local_shift) for hr in range(0,plot_len)]
            t_plot=[mpl.dates.date2num(hr) for hr in t[0:plot_len]]  
            
            row_num=len(var_names)
            col_num=1        
            fig, ax = plt.subplots(row_num,col_num)
            fig.set_figwidth(6.5*1.5*col_num) #190mm
            fig.set_figheight(6.5*0.75*row_num) #heigh/width=3/4
#             fig.set_dpi(300) 

            for k in range(row_num):
                var_name = var_names[k]
                if var_name in var_list:                    

                    var_value = f[var_name].values[:]
                    var_unit = var_units[var_names.index(var_name)]
                    
                    if var_unit == 'kg m-2 s-1':
                        var_value = var_value*86400 # P from kg/m^2/s to mm/day
                        var_unit = 'mm/d'
                    elif var_unit == 'kg m-2':
                        var_unit = 'mm'
                        var_value = var_value
                    elif var_unit == 'm s-1':
                        var_unit = 'mm/d'
                        var_value = var_value*1000*86400 
                    elif var_unit == 'K':
                        var_unit = '$^\circ$C'
                        var_value = var_value - 273.15 # temp from K to C
                        
                    var_value_plot = var_value[plot_start+0:plot_start+plot_len, hru_index] 
                    ax[k].plot_date(t_plot, var_value_plot, 'ko-', linewidth=1.0, markersize=1.0, alpha=0.8)   

                    title = var_name.replace('scalar', '')
                    ax[k].set_title(title, weight='semibold', fontsize='small')
                    
                    y_label = var_name.replace('scalar', '') + ' (' + var_unit + ')'
                    ax[k].set_ylabel(y_label, fontsize='small')
                    
                    del var_value, var_value_plot

            fig.tight_layout()
            plt.close(fig)                    
            pdf.savefig(fig)
    print('Done')    
f.close()
   

hru 1
2007-10-01 00:00:00
Done
hru 2
2007-10-01 00:00:00
Done
hru 3
2007-10-01 00:00:00
Done


NameError: name 'hru_index' is not defined